In [ ]:
# STEP 1: Install Kaggle API
!pip install -q kaggle

# STEP 2: Upload kaggle.json
from google.colab import files
files.upload()  # upload kaggle.json

# STEP 3: Move kaggle.json to ~/.kaggle
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# STEP 4: Download dataset
!kaggle datasets download -d sushantshetty/shabd-complete-hindi-characters-dataset --force

# STEP 5: Unzip dataset
!unzip -q shabd-complete-hindi-characters-dataset.zip -d dataset

In [2]:
import pandas as pd

# Load training and testing CSVs
train_df = pd.read_csv("dataset/train(grayscale).csv")
test_df = pd.read_csv("dataset/test(grayscale).csv")

# Check dataset shapes
print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)

# See first few rows
train_df.head()


Train shape: (243456, 1026)
Test shape: (60672, 1026)


,Index,label,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,...,f_1015,f_1016,f_1017,f_1018,f_1019,f_1020,f_1021,f_1022,f_1023,f_1024
0,83,अ,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
1,620,अ,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
2,508,अ,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
3,497,अ,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
4,679,अ,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255


In [2]:
# Drop "Index" column
train_df = train_df.drop(columns=["Index"])
test_df = test_df.drop(columns=["Index"])

# Separate features and labels
y_train = train_df["label"].values
X_train = train_df.drop(columns=["label"]).values

y_test = test_df["label"].values
X_test = test_df.drop(columns=["label"]).values

print("X_train shape:", X_train.shape)  # (243456, 1024)
print("y_train shape:", y_train.shape)
print("Unique labels:", np.unique(y_train)[:20])  # show a few characters


X_train shape: (243456, 1024)
y_train shape: (243456,)
Unique labels: ['अ' 'अं' 'अः' 'आ' 'इ' 'ई' 'उ' 'ऊ' 'ऋ' 'ए' 'ऐ' 'ओ' 'औ' 'क' 'कं' 'कः' 'का'
 'कि' 'की' 'कु']


In [3]:
# Reshape into 32x32 images
X_train = X_train.reshape(-1, 32, 32, 1).astype("float32") / 255.0
X_test = X_test.reshape(-1, 32, 32, 1).astype("float32") / 255.0

print("Reshaped X_train:", X_train.shape)  # (243456, 32, 32, 1)
print("Reshaped X_test:", X_test.shape)


Reshaped X_train: (243456, 32, 32, 1)
Reshaped X_test: (60672, 32, 32, 1)


In [4]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Convert Hindi characters → numeric labels
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

# One-hot encode for CNN
y_train_cat = to_categorical(y_train_encoded)
y_test_cat = to_categorical(y_test_encoded)

print("Classes:", le.classes_[:20])   # show first 20 Hindi characters
print("y_train_cat shape:", y_train_cat.shape)


Classes: ['अ' 'अं' 'अः' 'आ' 'इ' 'ई' 'उ' 'ऊ' 'ऋ' 'ए' 'ऐ' 'ओ' 'औ' 'क' 'कं' 'कः' 'का'
 'कि' 'की' 'कु']
y_train_cat shape: (243456, 384)


In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models

num_classes = y_train_cat.shape[1]  # 384

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(32,32,1)),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.Flatten(),

    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 30, 30, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 4, 4, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 384)            │        98,688 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 715,904 (2.73 MB)

 Trainable params: 715,904 (2.73 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
history = model.fit(
    X_train, y_train_cat,
    validation_data=(X_test, y_test_cat),
    epochs=10,
    batch_size=256
)


Epoch 1/10
951/951 ━━━━━━━━━━━━━━━━━━━━ 18s 11ms/step - accuracy: 0.3374 - loss: 3.2407 - val_accuracy: 0.9628 - val_loss: 0.1302
Epoch 2/10
951/951 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - accuracy: 0.9043 - loss: 0.2938 - val_accuracy: 0.9921 - val_loss: 0.0307
Epoch 3/10
951/951 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.9500 - loss: 0.1494 - val_accuracy: 0.9955 - val_loss: 0.0167
Epoch 4/10
951/951 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.9654 - loss: 0.1032 - val_accuracy: 0.9979 - val_loss: 0.0091
Epoch 5/10
951/951 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.9731 - loss: 0.0808 - val_accuracy: 0.9976 - val_loss: 0.0080
Epoch 6/10
951/951 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.9778 - loss: 0.0678 - val_accuracy: 0.9986 - val_loss: 0.0048
Epoch 7/10
951/951 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.9821 - loss: 0.0539 - val_accuracy: 0.9990 - val_loss: 0.0037
Epoch 8/10
951/951 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - accuracy: 0.9834 - loss: 0.0497 - val_accu

In [7]:
test_loss, test_acc = model.evaluate(X_test, y_test_cat, verbose=2)
print("Test accuracy:", test_acc)


1896/1896 - 5s - 3ms/step - accuracy: 0.9994 - loss: 0.0021
Test accuracy: 0.9993736743927002


In [8]:
model.save("hindi_char_cnn.h5")
print("Model saved successfully!")


Model saved successfully!


In [9]:
from tensorflow.keras.models import load_model
import numpy as np

# Load model
model = load_model("hindi_char_cnn.h5")

# Example: Predict on first test image
sample = X_test[0].reshape(1,32,32,1)
prediction = model.predict(sample)

predicted_class = np.argmax(prediction)
actual_class = np.argmax(y_test_cat[0])

print("Predicted:", le.inverse_transform([predicted_class])[0])
print("Actual:", le.inverse_transform([actual_class])[0])


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 614ms/step
Predicted: अ
Actual: अ


In [10]:
from google.colab import files
uploaded = files.upload()


Saving img1.jpg to img1.jpg
Saving img2.jpg to img2.jpg
Saving img3.jpg to img3.jpg
Saving img4.jpg to img4.jpg
Saving img5.jpg to img5.jpg


In [12]:
import pickle

# Save label encoder
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(le, f)

print("Label encoder saved!")


Label encoder saved!


In [14]:
from google.colab import files
uploaded = files.upload()
print(uploaded)


Saving img1.jpg to img1 (1).jpg
{'img1 (1).jpg': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00\x84\x00\x08\x08\x08\x08\t\x08\t\n\n\t\r\x0e\x0c\x0e\r\x13\x11\x10\x10\x11\x13\x1c\x14\x16\x14\x16\x14\x1c+\x1b\x1f\x1b\x1b\x1f\x1b+&.%#%.&D5//5DNB>BN_UU_wqw\x9c\x9c\xd1\x01\x08\x08\x08\x08\t\x08\t\n\n\t\r\x0e\x0c\x0e\r\x13\x11\x10\x10\x11\x13\x1c\x14\x16\x14\x16\x14\x1c+\x1b\x1f\x1b\x1b\x1f\x1b+&.%#%.&D5//5DNB>BN_UU_wqw\x9c\x9c\xd1\xff\xc2\x00\x11\x08\x05\x00\x02\xd0\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00,\x00\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x02\x03\x04\x05\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\xff\xda\x00\x0c\x03\x01\x00\x02\x10\x03\x10\x00\x00\x02\xcd\xb8\x89*\x8c\x88l\x12\xb3\xa2s\x03#I\x08\x1a\x01\x10d\x18\x8a\x81\xa0\xa7\x98\xea\x8c\xa8j\xc89\xd0i\xca&\xb2!\x13\x9d\x03$\x91\x8d\xc0\x1a\rgH6D\x11\xa8\xaa)\x08\xa3,\x9a\xb2\x90l\x04\x18\xc9\xb2\x89!\xa8\x8ddl\xe8\xa7\

In [16]:
import cv2

img_path = "img1.jpg"   # replace with actual uploaded file name
img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

if img is None:
    print("⚠️ Image not loaded. Check the file name!")
else:
    print("✅ Image loaded:", img.shape)


✅ Image loaded: (1280, 720)


In [17]:
if img is not None:
    img = cv2.resize(img, (32, 32))
    img = img.reshape(1, 32, 32, 1).astype("float32") / 255.0

    prediction = model.predict(img)
    pred_class = np.argmax(prediction)

    print("Predicted Letter:", le.inverse_transform([pred_class])[0])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 460ms/step
Predicted Letter: खी
